In [ ]:
# @title Setup
from google.colab import auth
# from google.cloud import bigquery
from google.colab import data_table
import networkx as nx
import pandas as pd


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from datetime import datetime, timedelta

# Links Data


In [ ]:
links_df = pd.read_parquet(f'/content/drive/My Drive/farcaster_bigquery/links/links.parquet')
# links_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/links.csv')

In [ ]:
links_df = links_df[links_df.type.isin(['follow', 'unfollow'])]
links_df = links_df.drop_duplicates(['target_fid', 'fid'], keep=False)
# links_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/csv/links_followsOnly.csv')
# links_df.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/parquet/links_followsOnly.parquet')

In [ ]:
links_df.head()

,fid,timestamp,target_fid,type,deleted_at
0,19099,2024-04-20 22:06:08+00:00,461009,follow,NaT
1,252634,2024-05-03 10:06:31+00:00,250833,follow,NaT
2,418937,2024-04-06 09:26:16+00:00,415186,follow,NaT
3,385938,2024-04-17 19:43:30+00:00,403156,follow,NaT
4,500406,2024-04-29 11:21:25+00:00,484567,follow,NaT


In [ ]:
G = nx.from_pandas_edgelist(links_df, source='fid', target='target_fid', create_using=nx.DiGraph)

In [ ]:
# prompt: calculate HITS and Trustrank using the networkx digraph 'g'


# Calculate HITS
hubs, authorities = nx.hits(G)

# Calculate TrustRank (assuming a small set of trusted nodes)
trusted_nodes = list(G.nodes())[:10]  # Replace with your trusted nodes
trustrank = nx.pagerank(G, personalization={node: 1 / len(trusted_nodes) for node in trusted_nodes})

# Print or use the calculated values
print("HITS Hub Scores:", hubs)
print("HITS Authority Scores:", authorities)
print("TrustRank Scores:", trustrank)


Output hidden; open in https://colab.research.google.com to view.

In [16]:
# prompt: insert the hubs, authorities and trustrank of a dataframe, insert FID as the first column

import pandas as pd

# Create DataFrames from the calculated values
hubs_df = pd.DataFrame.from_dict(hubs, orient='index', columns=['hub_score'])
authorities_df = pd.DataFrame.from_dict(authorities, orient='index', columns=['authority_score'])
trustrank_df = pd.DataFrame.from_dict(trustrank, orient='index', columns=['trustrank_score'])

# Reset index to make FID a column
hubs_df = hubs_df.reset_index().rename(columns={'index': 'fid'})
authorities_df = authorities_df.reset_index().rename(columns={'index': 'fid'})
trustrank_df = trustrank_df.reset_index().rename(columns={'index': 'fid'})

# Merge the DataFrames on FID
merged_df = pd.merge(hubs_df, authorities_df, on='fid', how='outer')
merged_df = pd.merge(merged_df, trustrank_df, on='fid', how='outer')

print(merged_df)


           fid  hub_score  authority_score  trustrank_score
0        19099   0.000009     4.094427e-05     1.570311e-02
1       461009   0.000005     1.389252e-05     1.568442e-02
2       252634   0.000003     9.061602e-06     1.572188e-02
3       250833   0.000001     2.145580e-06     1.568568e-02
4       418937   0.000004     7.854914e-06     1.568001e-02
...        ...        ...              ...              ...
410099  189247  -0.000000     4.250476e-08     2.932444e-09
410100  185724  -0.000000     4.250476e-08     2.932444e-09
410101  164852  -0.000000     4.250476e-08     2.932444e-09
410102  248489  -0.000000     4.250476e-08     2.932444e-09
410103  174973  -0.000000     4.250476e-08     2.932444e-09

[410104 rows x 4 columns]


In [14]:
df_merged = df_merged.sort_index()

In [17]:
df_merged.to_csv(f'/content/drive/My Drive/farcaster_bigquery/links/analytics/fid_trust_ranks.csv')
df_merged.to_parquet(f'/content/drive/My Drive/farcaster_bigquery/links/analytics/fid_trust_ranks.parquet')